In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from smooth.adam_general.core.adam import ADAM

import numpy as np
import pandas as pd

In [3]:
from typing import List, Union, Dict, Any
from smooth.adam_general._adam_general import adam_fitter, adam_forecaster
from numpy.linalg import eigvals
import nlopt

In [4]:
#### Global level and ETS(A,N,N) ####
np.random.seed(33)  # For reproducibility
n_points = 100
time_series = np.random.normal(100, 10, n_points)  # Random errors
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [5]:
ts_df

,value
2023-01-31,96.811465
2023-02-28,83.970194
2023-03-31,84.647821
2023-04-30,94.295991
2023-05-31,97.832717
...,...
2030-12-31,101.952350
2031-01-31,105.358388
2031-02-28,106.842993
2031-03-31,128.212793


In [6]:
np.random.seed(65)  # For reproducibility
n_points = 39  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)

/tmp/ipykernel_15856/1138340368.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency


In [13]:
import time

start_time = time.time()

model = "MAM"
lags = [12]

np.random.seed(33)  # For reproducibility
n_points = 39  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)

adam = ADAM(model, lags)
adam.fit(ts_df)
#print(adam.adam_created)
fc = adam.predict(h = 12)
#print(adam.adam_created)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 0.1842 seconds


,mean,lower_0.025,upper_0.975
2026-04-30,2342.291208,858.769709,4556.999101
2026-05-31,2442.739216,460.210057,6047.137982
2026-06-30,2519.506769,231.960794,7467.835382
2026-07-31,2304.476588,93.049214,7940.217629
2026-08-31,2074.328459,31.666160,8159.406488
2026-09-30,1981.041562,9.508047,8784.600552
2026-10-31,1715.512994,2.077173,8493.884373
2026-11-30,1486.656372,0.350776,8154.801032
2026-12-31,1275.590024,0.043404,7699.706378
2027-01-31,1067.546154,0.003690,7047.766157


In [14]:
import time

start_time = time.time()

model = "MAM"
lags = [12]

np.random.seed(33)  # For reproducibility
n_points = 39  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)

adam = ADAM(model, lags)
adam.fit(ts_df)
#print(adam.adam_created)
fc = adam.predict(h = 12)
#print(adam.adam_created)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 0.1905 seconds


,mean,lower_0.025,upper_0.975
2026-04-30,2342.291208,858.769709,4556.999101
2026-05-31,2442.739216,460.210057,6047.137982
2026-06-30,2519.506769,231.960794,7467.835382
2026-07-31,2304.476588,93.049214,7940.217629
2026-08-31,2074.328459,31.666160,8159.406488
2026-09-30,1981.041562,9.508047,8784.600552
2026-10-31,1715.512994,2.077173,8493.884373
2026-11-30,1486.656372,0.350776,8154.801032
2026-12-31,1275.590024,0.043404,7699.706378
2027-01-31,1067.546154,0.003690,7047.766157


In [17]:
# Load the AirPassengers dataset from Python
import pandas as pd
import numpy as np
# Create the AirPassengers dataset manually
data = [
    112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118,
    115, 126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140,
    145, 150, 178, 163, 172, 178, 199, 199, 184, 162, 146, 166,
    171, 180, 193, 181, 183, 218, 230, 242, 209, 191, 172, 194,
    196, 196, 236, 235, 229, 243, 264, 272, 237, 211, 180, 201,
    204, 188, 235, 227, 234, 264, 302, 293, 259, 229, 203, 229,
    242, 233, 267, 269, 270, 315, 364, 347, 312, 274, 237, 278,
    284, 277, 317, 313, 318, 374, 413, 405, 355, 306, 271, 306,
    315, 301, 356, 348, 355, 422, 465, 467, 404, 347, 305, 336,
    340, 318, 362, 348, 363, 435, 491, 505, 404, 359, 310, 337,
    360, 342, 406, 396, 420, 472, 548, 559, 463, 407, 362, 405,
    417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390, 432
]

# Create a proper datetime index
dates = pd.date_range(start='1949-01-01', periods=len(data), freq='MS')

# Create a pandas Series with the data
air_passengers_series = pd.Series(data, index=dates, name='AirPassengers')

# Create a DataFrame with the time series
ts_df = pd.DataFrame({'value': air_passengers_series})

print("AirPassengers dataset loaded:")
print(ts_df.head())



AirPassengers dataset loaded:
            value
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121


In [9]:
# problem is the selector!!
# lets fix the seelctor!!

In [18]:
import time

start_time = time.time()

model = "ZXZ"
lags = [12]


adam = ADAM(model, lags)
adam.fit(ts_df)
fc = adam.predict(h = 12)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 2.3591 seconds


,mean,lower_0.025,upper_0.975
1961-01-01,446.745880,412.661507,482.163463
1961-02-01,421.460877,387.180603,457.174452
1961-03-01,475.544201,434.615693,518.288047
1961-04-01,499.710761,454.469776,547.067557
1961-05-01,512.928279,464.314130,563.927971
1961-06-01,585.301208,527.458097,646.110430
1961-07-01,675.128957,605.790786,748.169926
1961-08-01,666.561737,595.620269,741.436966
1961-09-01,553.760531,492.836822,618.183254
1961-10-01,491.464745,435.691134,550.548758


In [10]:
import time

start_time = time.time()

model = "ZXZ"
lags = [12]


adam = ADAM(model, lags)
adam.fit(ts_df)
fc = adam.predict(h = 12)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 2.3237 seconds


,mean,lower_0.025,upper_0.975
1961-01-01,446.745880,412.661507,482.163463
1961-02-01,421.460877,387.180603,457.174452
1961-03-01,475.544201,434.615693,518.288047
1961-04-01,499.710761,454.469776,547.067557
1961-05-01,512.928279,464.314130,563.927971
1961-06-01,585.301208,527.458097,646.110430
1961-07-01,675.128957,605.790786,748.169926
1961-08-01,666.561737,595.620269,741.436966
1961-09-01,553.760531,492.836822,618.183254
1961-10-01,491.464745,435.691134,550.548758


In [19]:
adam = ADAM(model="ANN", lags=[12])
adam.fit(ts_df)

preds = adam.predict(h = 12)
preds

,mean,lower_0.025,upper_0.975
1961-01-01,432.0,365.461874,498.538126
1961-02-01,432.0,337.900880,526.099120
1961-03-01,432.0,316.752586,547.247414
1961-04-01,432.0,298.923749,565.076251
1961-05-01,432.0,283.216228,580.783772
1961-06-01,432.0,269.015544,594.984456
1961-07-01,432.0,255.956667,608.043333
1961-08-01,432.0,243.801761,620.198239
1961-09-01,432.0,232.385623,631.614377
1961-10-01,432.0,221.587972,642.412028


In [20]:
adam = ADAM(model="ANN", lags=[12])
adam.fit(ts_df)

preds = adam.predict(h = 12)
preds

,mean,lower_0.025,upper_0.975
1961-01-01,432.0,365.461874,498.538126
1961-02-01,432.0,337.900880,526.099120
1961-03-01,432.0,316.752586,547.247414
1961-04-01,432.0,298.923749,565.076251
1961-05-01,432.0,283.216228,580.783772
1961-06-01,432.0,269.015544,594.984456
1961-07-01,432.0,255.956667,608.043333
1961-08-01,432.0,243.801761,620.198239
1961-09-01,432.0,232.385623,631.614377
1961-10-01,432.0,221.587972,642.412028


In [22]:
#### Local level and ETS(A,N,N) ####
np.random.seed(41)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [23]:
# ETS(A,N,N) with alpha=0.1
model = ADAM(model="ANN", lags=[12], persistence=0.1)
model.fit(ts_df)
print("ETS(A,N,N) with alpha=0.1:")
print(model.predict(h = 12))

# ETS(A,N,N) with estimated alpha 
model = ADAM(model="ANN", lags=[12])
model.fit(ts_df)
print("\nETS(A,N,N) with estimated parameters:")
print(model.adam_estimated["B"])
print(model.predict(h = 12))

# ETS(M,N,N) with estimated alpha
model = ADAM(model="MNN", lags=[12], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,N,N) with estimated parameters:")
print(model.predict(h = 12))


ETS(A,N,N) with alpha=0.1:
                 mean  lower_0.025  upper_0.975
2033-01-31  83.225986    62.908868   103.543104
2033-02-28  83.225986    62.807536   103.644436
2033-03-31  83.225986    62.706703   103.745269
2033-04-30  83.225986    62.606364   103.845608
2033-05-31  83.225986    62.506510   103.945462
2033-06-30  83.225986    62.407136   104.044836
2033-07-31  83.225986    62.308233   104.143739
2033-08-31  83.225986    62.209796   104.242176
2033-09-30  83.225986    62.111818   104.340154
2033-10-31  83.225986    62.014293   104.437680
2033-11-30  83.225986    61.917213   104.534759
2033-12-31  83.225986    61.820574   104.631398

ETS(A,N,N) with estimated parameters:
[6.53178667e-02 9.69906981e+01]
                 mean  lower_0.025  upper_0.975
2033-01-31  84.708881    64.380865   105.036897
2033-02-28  84.708881    64.337547   105.080215
2033-03-31  84.708881    64.294322   105.123441
2033-04-30  84.708881    64.251187   105.166575
2033-05-31  84.708881    64.208143   1

In [24]:
#### Local trend and ETS(A,A,N) ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))


In [25]:
# ETS(A,A,N) with fixed parameters
model = ADAM(model="AAN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df)
print("ETS(A,A,N):")
print(model.predict(h=12))

# ETS(A,Ad,N) with estimated parameters
model = ADAM(model="AAdN", lags=[12])
model.fit(ts_df)
print("\nETS(A,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(A,Ad,N) with fixed parameters
model = ADAM(model="AAdN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df)
print("\nETS(A,Ad,N):")
print(model.predict(h=12))

ETS(A,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.255525   145.244436   185.266614
2033-02-28  165.763955   145.609301   185.918609
2033-03-31  166.272384   145.923950   186.620819
2033-04-30  166.780814   146.182025   187.379604
2033-05-31  167.289244   146.377895   188.200593
2033-06-30  167.797674   146.506778   189.088570
2033-07-31  168.306104   146.564813   190.047395
2033-08-31  168.814534   146.549105   191.079962
2033-09-30  169.322963   146.457722   192.188205
2033-10-31  169.831393   146.289642   193.373145
2033-11-30  170.339823   146.044679   194.634967
2033-12-31  170.848253   145.723376   195.973130

ETS(A,Ad,N) estimated parameters:
[ 1.68275229e-01  1.04248642e-02  9.86301308e-01  1.04774858e+02
 -3.80921118e-01]

ETS(A,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.235667   143.517496   182.953839
2033-02-28  163.701313   143.844309   183.558318
2033-03-31  164.140767   144.101681   184.179852
2033-04-30  164.555501 

In [26]:
# ETS(M,A,N) with fixed parameters
model = ADAM(model="MAN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("ETS(M,A,N):")
print(model.predict(h=12))

# ETS(M,Ad,N) with estimated parameters
model = ADAM(model="MAdN", lags=[12], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Ad,N) with fixed parameters
model = ADAM(model="MAdN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Ad,N):")
print(model.predict(h=12))

ETS(M,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.256254   137.193633   193.318875
2033-02-28  165.764546   137.413662   194.115429
2033-03-31  166.272837   137.561600   194.984074
2033-04-30  166.781129   137.627796   195.934462
2033-05-31  167.289421   137.603528   196.975315
2033-06-30  167.797713   137.481177   198.114249
2033-07-31  168.306005   137.254365   199.357644
2033-08-31  168.814297   136.918028   200.710565
2033-09-30  169.322588   136.468434   202.176743
2033-10-31  169.830880   135.903132   203.758628
2033-11-30  170.339172   135.220863   205.457481
2033-12-31  170.847464   134.421414   207.273514

ETS(M,Ad,N) estimated parameters:
[ 1.49690928e-01  1.32751821e-02  9.67541206e-01  1.05756209e+02
 -4.62346860e-01]

ETS(M,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.195224   135.642594   190.747854
2033-02-28  163.659519   135.834075   191.484963
2033-03-31  164.096847   135.941610   192.252085
2033-04-30  164.508774 

In [27]:
# ETS(M,M,N) with fixed parameters
model = ADAM(model="MMN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("ETS(M,M,N):")
print(model.adam_estimated["B"])
print(model.predict(h=12))

# ETS(M,Md,N) with estimated parameters
model = ADAM(model="MMdN", lags=[12], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Md,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Md,N) with fixed parameters
model = ADAM(model="MMdN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Md,N):")
print(model.predict(h=12))


ETS(M,M,N):
[107.91006209   0.9932759 ]
                  mean  lower_0.025  upper_0.975
2033-01-31  165.593071   137.459506   193.726637
2033-02-28  166.074017   137.656317   194.491718
2033-03-31  166.556360   137.782104   195.330617
2033-04-30  167.040104   137.827226   196.252982
2033-05-31  167.525253   137.782976   197.267530
2033-06-30  168.011811   137.641751   198.381870
2033-07-31  168.499782   137.397191   199.602372
2033-08-31  168.989170   137.044250   200.934091
2033-09-30  169.479980   136.579208   202.380752
2033-10-31  169.972215   135.999630   203.944800
2033-11-30  170.465880   135.304264   205.627495
2033-12-31  170.960978   134.492904   207.429052

ETS(M,Md,N) estimated parameters:
[1.49082653e-01 1.27970575e-02 9.65242357e-01 1.06257974e+02
 9.94932936e-01]

ETS(M,Md,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.174599   135.701118   190.648080
2033-02-28  163.629296   135.885709   191.372883
2033-03-31  164.056585   135.986882   192.126287


/home/filtheo/smooth/python/smooth/adam_general/core/forecaster.py:342: UserWarning: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful.
  warnings.warn("Your model has a potentially explosive multiplicative trend. "
